In [1]:

#imports
import os
import glob
import pandas as pd
import random
import numpy as np
from datetime import datetime
import csv
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,DBSCAN
from sklearn.preprocessing import StandardScaler, Normalizer

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals.six import StringIO
import subprocess

import seaborn as sns
from IPython.display import Image
import pydotplus
from sklearn.tree import export_graphviz

from sklearn.tree import DecisionTreeClassifier

C:\Users\MMscichowski\.conda\envs\metis\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
REdata = pd.read_csv("data\REdata.csv",delimiter='*') 
# Preview the first 5 lines of the loaded data 
REdata.head()

C:\Users\MMscichowski\.conda\envs\metis\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,6,7,11,19,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ADDRESS,APARTMENT_NUMBER,BLOCK,BOROUGH,BUILDING_CLASS_AS_OF_FINAL_ROLL_17/18,BUILDING_CLASS_AS_OF_FINAL_ROLL_18/19,BUILDING_CLASS_AT_PRESENT,BUILDING_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_CATEGORY,...,TAX_CLASS_AT_PRESENT,TAX_CLASS_AT_TIME_OF_SALE,TOTAL_UNITS,YEAR_BUILT,ZIP_CODE,SALE_DATEadj,SALE_YEAR,counter,NEW,DECADE_BUILT
0,0,96 AVENUE B,,402,1,NaN,NaN,S1,S1,01 ONE FAMILY DWELLINGS,...,1,1,2,1901.0,10009,2014-12-05,2014.0,1,False,"(1900.0, 1910.0]"
1,1,17 AVENUE D,,372,1,NaN,NaN,S2,S2,02 TWO FAMILY DWELLINGS,...,1,1,3,1910.0,10002,2014-01-31,2014.0,1,False,"(1900.0, 1910.0]"
2,2,243 EAST 7TH STREET,,377,1,NaN,NaN,C0,C0,03 THREE FAMILY DWELLINGS,...,1,1,3,1899.0,10009,2014-11-26,2014.0,1,False,"(1890.0, 1900.0]"
3,3,300 EAST 2ND STREET,,372,1,NaN,NaN,C7,C7,07 RENTALS - WALKUP APARTMENTS,...,2B,2,10,1920.0,10009,2014-07-18,2014.0,1,False,"(1910.0, 1920.0]"
4,4,324 EAST 4TH STREET,,373,1,NaN,NaN,D1,C3,07 RENTALS - WALKUP APARTMENTS,...,2,2,11,1920.0,10009,2014-05-30,2014.0,1,False,"(1910.0, 1920.0]"


In [3]:
REdata.dtypes

Unnamed: 0                                 int64
ADDRESS                                   object
APARTMENT_NUMBER                          object
BLOCK                                      int64
BOROUGH                                    int64
BUILDING_CLASS_AS_OF_FINAL_ROLL_17/18     object
BUILDING_CLASS_AS_OF_FINAL_ROLL_18/19     object
BUILDING_CLASS_AT_PRESENT                 object
BUILDING_CLASS_AT_TIME_OF_SALE            object
BUILDING_CLASS_CATEGORY                   object
COMMERCIAL_UNITS                         float64
EASE-MENT                                 object
GROSS_SQUARE_FEET                        float64
LAND_SQUARE_FEET                          object
LOT                                        int64
NEIGHBORHOOD                              object
RESIDENTIAL_UNITS                        float64
SALE_DATE                                 object
SALE_PRICE                               float64
TAX_CLASS_AS_OF_FINAL_ROLL_17/18          object
TAX_CLASS_AS_OF_FINA

In [4]:
REmask = ((REdata['NEW']==False)  &(REdata['SALE_YEAR']==2016)  & (REdata['RESIDENTIAL_UNITS']==1)  & (REdata['SALE_PRICE'] >20) & (REdata['COMMERCIAL_UNITS'] <2) & (REdata['YEAR_BUILT'] >0)& (REdata['YEAR_BUILT'] <3995)& (REdata['ZIP_CODE'] >10000) & (REdata['ZIP_CODE'] <1000000))
REmaskDF = REdata[REmask]
#REmaskDF2 = REmaskDF[['YEAR_BUILT',"SALE_PRICE"]]

group_zipmedian = REmaskDF.groupby(['ZIP_CODE','SALE_YEAR'])['SALE_PRICE'].median() 

print(group_zipmedian)

#group_zipmedian.to_csv('data\zipmedians2016.csv', sep='*')

ZIP_CODE  SALE_YEAR
10001     2016.0       1637500.0
10002     2016.0       1130000.0
10003     2016.0       1825000.0
10004     2016.0       1167500.0
10005     2016.0       1385000.0
10006     2016.0        755000.0
10007     2016.0       3718172.0
10009     2016.0        950000.0
10010     2016.0       2475000.0
10011     2016.0       3225000.0
10012     2016.0       2890000.0
10013     2016.0       4241547.0
10014     2016.0       2902012.0
10016     2016.0       1237500.0
10017     2016.0        940000.0
10018     2016.0       1767500.0
10019     2016.0       1575127.0
10021     2016.0       1680000.0
10022     2016.0       2380000.0
10023     2016.0       1950000.0
10024     2016.0       1435000.0
10025     2016.0        992000.0
10026     2016.0        945000.0
10027     2016.0        969000.0
10028     2016.0       1600000.0
10029     2016.0        762061.5
10030     2016.0        771000.0
10031     2016.0        640000.0
10032     2016.0        503001.0
10033     2016.0       

In [5]:
REmask = ((REdata['NEW']==False)   & (REdata['RESIDENTIAL_UNITS']==1)  & (REdata['SALE_PRICE'] >20) & (REdata['COMMERCIAL_UNITS'] <2) & (REdata['YEAR_BUILT'] >0)& (REdata['YEAR_BUILT'] <3995)& (REdata['ZIP_CODE'] >10000) & (REdata['ZIP_CODE'] <1000000))
REmaskDF = REdata[REmask]
#REmaskDF2 = REmaskDF[['YEAR_BUILT',"SALE_PRICE"]]
#&(REdata['SALE_YEAR']==2016) 
group_zipmedian = REmaskDF.groupby(['ZIP_CODE','SALE_YEAR'])['SALE_PRICE'].median() 

print(group_zipmedian)

#group_zipmedian.to_csv('data\zipmedians2016b.csv', sep='*')

ZIP_CODE  SALE_YEAR
10001     2014.0       1275000.0
          2015.0       1825000.0
          2016.0       1637500.0
          2017.0       1905000.0
          2018.0       2592618.5
10002     2014.0       1082500.0
          2015.0       1665056.0
          2016.0       1130000.0
          2017.0        933249.5
          2018.0       1846850.5
10003     2014.0       1660000.0
          2015.0       1880000.0
          2016.0       1825000.0
          2017.0       2375000.0
          2018.0       1787500.0
10004     2014.0       1073500.0
          2015.0       1181170.0
          2016.0       1167500.0
          2017.0       1200000.0
          2018.0       1193250.0
10005     2014.0       1145000.0
          2015.0       1112500.0
          2016.0       1385000.0
          2017.0       1302500.0
          2018.0       1169000.0
10006     2014.0        977500.0
          2015.0        774050.0
          2016.0        755000.0
          2017.0       2563444.0
          2018.0       

In [6]:
group_zipmedian = REmaskDF.groupby(['NEIGHBORHOOD','SALE_YEAR'])['SALE_PRICE'].median() 

print(group_zipmedian)
group_zipmedian.to_csv('data\zipmedians3.csv', sep='*')

NEIGHBORHOOD            SALE_YEAR
ALPHABETCITY            2014.0       1068000.0
                        2015.0        931000.0
                        2016.0        992440.0
                        2017.0       1185950.0
                        2018.0       1016213.0
CHELSEA                 2014.0       1760000.0
                        2015.0       1735000.0
                        2016.0       2169050.0
                        2017.0       1915000.0
                        2018.0       1923712.5
CHINATOWN               2014.0       1750000.0
                        2015.0       1635000.0
                        2016.0       1355000.0
                        2017.0       2050000.0
                        2018.0        812500.0
CIVICCENTER             2014.0       2156908.0
                        2015.0       2900000.0
                        2016.0       4267090.0
                        2017.0       5704950.0
                        2018.0       4395000.0
CLINTON                 20

C:\Users\MMscichowski\.conda\envs\metis\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [7]:
group_zipmedian = REmaskDF.groupby(['SALE_YEAR'])['SALE_PRICE'].median() 
print(group_zipmedian)

SALE_YEAR
2014.0    1360000.0
2015.0    1480734.0
2016.0    1671866.0
2017.0    1645000.0
2018.0    1585000.0
Name: SALE_PRICE, dtype: float64
